<a href="https://colab.research.google.com/github/henrijoel1209/FORUM/blob/main/Exercice3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
Exercice 3: Manipulation de fichiers
Création et lecture de fichiers CSV avec calcul de densité
"""

import pandas as pd
import os
from google.colab import drive

def creer_fichier_regions():
    """Crée le fichier regions.csv avec des données réelles"""
    try:
        drive.mount('/content/drive', force_remount=False)

        base_path = '/content/drive/My Drive/Exercice/'

        # Charger les données réelles
        df_pop = pd.read_excel(base_path + 'DATA/Populations_CI_2019.xlsx', sheet_name='CIV_ADM1_POP')
        df_pop = df_pop.dropna(subset=['ADM1_FR'])
        df_pop = df_pop[df_pop['ADM1_FR'] != '#adm1+name']
        df_pop['T_TL'] = pd.to_numeric(df_pop['T_TL'], errors='coerce')

        # Charger les surfaces
        import json
        with open(base_path + 'DATA/extracted/civ_admin1.geojson', 'r', encoding='utf-8') as f:
            geo_data = json.load(f)

        surface_data = []
        for feature in geo_data['features']:
            props = feature['properties']
            surface_data.append({
                'region': props['adm1_name'],
                'surface': props['area_sqkm']
            })

        df_surface = pd.DataFrame(surface_data)

        # Fusionner les données
        df_final = pd.merge(df_pop, df_surface, left_on='ADM1_FR', right_on='region')

        # Créer le fichier regions.csv
        regions_df = df_final[['ADM1_FR', 'T_TL', 'surface']].copy()
        regions_df.columns = ['Nom', 'Population', 'Superficie']

        # Prendre les 10 premières régions pour simplifier
        regions_df = regions_df.head(10)

        # Sauvegarder en CSV
        csv_path = base_path + 'regions.csv'
        regions_df.to_csv(csv_path, index=False, encoding='utf-8')

        print(f"✅ Fichier regions.csv créé avec {len(regions_df)} régions")
        print(f"📍 Emplacement: {csv_path}")

        return csv_path

    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None

def lire_et_calculer_densite(csv_path):
    """Lit le fichier regions.csv et calcule les densités"""
    try:
        # Lire le fichier CSV
        df = pd.read_csv(csv_path, encoding='utf-8')

        print(f"\n📂 Lecture du fichier regions.csv")
        print(f"📊 {len(df)} régions trouvées")

        # Calculer la densité
        df['Densité'] = df['Population'] / df['Superficie']

        # Ajouter la colonne Couleur (extension)
        df['Couleur'] = df['Densité'].apply(classifier_couleur)

        # Créer le fichier densites.csv
        densites_df = df[['Nom', 'Densité', 'Couleur']].copy()

        output_path = csv_path.replace('regions.csv', 'densites.csv')
        densites_df.to_csv(output_path, index=False, encoding='utf-8')

        print(f"✅ Fichier densites.csv créé")
        print(f"📍 Emplacement: {output_path}")

        return densites_df

    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None

def classifier_couleur(densite):
    """Classifie la densité avec une couleur"""
    if densite < 50:
        return "verte"
    elif densite < 150:
        return "jaune"
    else:
        return "rouge"

def afficher_resultats(df):
    """Affiche les résultats de manière claire"""
    print("\n" + "="*60)
    print("RÉSULTATS - DENSITÉS PAR RÉGION")
    print("="*60)

    # Afficher les noms des colonnes pour débogage
    print("Colonnes disponibles:", df.columns.tolist())
    print()

    for index, row in df.iterrows():
        couleur_emoji = {
            'verte': '🟢',
            'jaune': '🟡',
            'rouge': '🔴'
        }

        emoji = couleur_emoji.get(row['Couleur'], '⚪')

        print(f"\n{index + 1}. {row['Nom']}")
        print(f"   Densité: {row['Densité']:,.2f} hab/km²")
        print(f"   Classification: {emoji} {row['Couleur']}")

def main():
    """Fonction principale de l'exercice 3"""
    print("="*60)
    print("EXERCICE 3: MANIPULATION DE FICHIERS")
    print("Création et traitement de fichiers CSV")
    print("="*60)

    # ÉTAPE 1: Créer le fichier regions.csv
    print("\n📝 ÉTAPE 1: Création du fichier regions.csv")
    csv_path = creer_fichier_regions()

    if csv_path is None:
        print("❌ Impossible de créer le fichier regions.csv")
        return

    # ÉTAPE 2: Lire et calculer les densités
    print("\n📊 ÉTAPE 2: Calcul des densités")
    densites_df = lire_et_calculer_densite(csv_path)

    if densites_df is None:
        print("❌ Impossible de calculer les densités")
        return

    # ÉTAPE 3: Afficher les résultats
    print("\n📋 ÉTAPE 3: Affichage des résultats")
    afficher_resultats(densites_df)

    # Résumé
    print("\n" + "="*60)
    print("RÉSUMÉ DES OPÉRATIONS")
    print("="*60)
    print("✅ Fichier regions.csv créé avec colonnes: Nom, Population, Superficie")
    print("✅ Fichier densites.csv créé avec colonnes: Nom, Densité, Couleur")
    print("✅ Densités calculées: Population / Superficie")
    print("✅ Classification couleur: verte (<50), jaune (50-150), rouge (>150)")

if __name__ == "__main__":
    main()

EXERCICE 3: MANIPULATION DE FICHIERS
Création et traitement de fichiers CSV

📝 ÉTAPE 1: Création du fichier regions.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Fichier regions.csv créé avec 10 régions
📍 Emplacement: /content/drive/My Drive/Exercice/regions.csv

📊 ÉTAPE 2: Calcul des densités

📂 Lecture du fichier regions.csv
📊 10 régions trouvées
✅ Fichier densites.csv créé
📍 Emplacement: /content/drive/My Drive/Exercice/densites.csv

📋 ÉTAPE 3: Affichage des résultats

RÉSULTATS - DENSITÉS PAR RÉGION
Colonnes disponibles: ['Nom', 'Densité', 'Couleur']


1. Gbokle
   Densité: 73.74 hab/km²
   Classification: 🟡 jaune

2. Nawa
   Densité: 123.36 hab/km²
   Classification: 🟡 jaune

3. San Pedro
   Densité: 76.79 hab/km²
   Classification: 🟡 jaune

4. Indenie-Djuablin
   Densité: 92.56 hab/km²
   Classification: 🟡 jaune

5. Sud-Comoe
   Densité: 103.67 hab/km²
   Classification: 🟡 jaune

6. Folon
   